In [1]:
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cf
import numpy as np
import xarray as xr
import time
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES)

In [2]:
# Open the NetCDF file
ncfile = Dataset("../Rain.2023-09-25.nc")
rain = getvar(ncfile, "RAINNC", timeidx=ALL_TIMES)

# Get the latitude and longitude points
lats, lons = latlon_coords(rain)

# Get the cartopy mapping object
cart_proj = get_cartopy(rain)
xlim = cartopy_xlim(rain)
ylim = cartopy_ylim(rain)

dataset = xr.open_dataset("../NOV02_forecast/data/Rain.2023-11-02.nc")
pr = dataset['RAINNC']

dataset = xr.open_dataset('masking/mask.nc')
mask = dataset['__xarray_dataarray_variable__']
m = to_np(mask).astype('float')
m[m==0] = np.nan

In [3]:
# Create precip color map
nws_precip_colors = [
    "#fdfdfd",  # 10.00+
    "#04e9e7",  # 0.01 - 0.10 inches
    "#019ff4",  # 0.10 - 0.25 inches
    "#0300f4",  # 0.25 - 0.50 inches
    "#02fd02",  # 0.50 - 0.75 inches
    "#01c501",  # 0.75 - 1.00 inches
    "#008e00",  # 1.00 - 1.50 inches
    "#fdf802",  # 1.50 - 2.00 inches
    "#e5bc00",  # 2.00 - 2.50 inches
    "#fd9500",  # 2.50 - 3.00 inches
    "#fd0000",  # 3.00 - 4.00 inches
    "#d40000",  # 4.00 - 5.00 inches
    "#bc0000",  # 5.00 - 6.00 inches
    "#f800fd",  # 6.00 - 8.00 inches
    "#9854c6"  # 8.00 - 10.00 inches
]
precip_colormap = mpl.colors.ListedColormap(nws_precip_colors)
#YlGnBu = ["#fdfdfd","#ffffd9","#edf8b1","#c7e9b4","#7fcdbb","#41b6c4","#1d91c0","#225ea8","#253494","#081d58"]
#prob_colors = ["#fdfdfd","#ffffd9","#7fcdbb","#225ea8","#f7fcb9","#addd8e","#31a354","#ffffb2","#fecc5c","#fd8d3c","#f03b20","#bd0026","#fa9fb5","#c51b8a"]
#precip_colormap = mpl.colormaps["tab20c"]
#precip_colormap = mpl.colors.ListedColormap(prob_colors)
#jet = mpl.colormaps.get_cmap('jet')
#jet_colors = jet(np.linspace(0, 1, 14))
#print('jet.colors', jet_colors)
#white = [1.,1.,1.,1.]
#precip_colormap = tuple(np.vstack([white,jet_colors]))
#print(precip_colormap)

In [5]:
# Create a figure    
def plot(prec,r,thres,t,unit,levels):
    fig = plt.figure(figsize=(8,6))
    # Set the GeoAxes to the projection used by WRF
    ax = plt.axes(projection=cart_proj)

    norm = mpl.colors.BoundaryNorm(levels, 15)

    # Download and add the states and coastlines
    ax.add_feature(cf.BORDERS)
    ax.add_feature(cf.STATES)
    ax.coastlines('10m', linewidth=0.8)

#    plt.contourf(to_np(lons), to_np(lats), to_np(prec), levels,
    plt.pcolormesh(lons, lats, prec,
             transform=crs.PlateCarree(),
             cmap=precip_colormap, norm=norm)
    # Add a color bar
    plt.colorbar(ax=ax, shrink=.9, ticks = levels)

    # Set the map bounds
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Add the gridlines
    ax.gridlines(color='black', linestyle='dotted')

    plt.title(str(r)+' of rainfall > '+str(thres)+' mm/week at week '+str(t+1)+unit)

    #plt.show()
    plt.savefig('1102/P_t'+str(t+1)+'_e'+str(r)+'_'+str(thres)+'mm.png', dpi=200)
    plt.close()

In [10]:
for t in range(0,6):
#    for thres in [1, 5, 10, 25, 50]:
        thres = 1
        print((t+1)*1000+thres)
        prec = pr.isel(Time=t)*m
        prob = xr.where(prec>thres,100,0)
        print('plot pcolormesh')
        levels = [0, 1, 2, 4, 6, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100]
        start_time = time.time()
        #plot(prob.mean(dim='record',skipna=True),'Probability',thres,t,' (%)',levels)
        emean = xr.where(prec>thres,prec,np.nan)
        levels = [0, 1, 2, 5, 10, 15, 20, 25, 30, 40, 50, 60, 80, 100, 150, 200]
        #plot(emean.median(dim='record', skipna=True),'Median',thres,t,' (mm/week)',levels)
        #plot(emean.max(dim='record', skipna=True),'Max',thres,t,' (mm/week)',levels)
        plot(emean.mean(dim='record', skipna=True),'Mean',thres,t,' (mm/week)',levels)
        print("--- %.2f seconds ---" % (time.time() - start_time))
        print('done')

1001
plot pcolormesh


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)


--- 1.68 seconds ---
done
2001
plot pcolormesh


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)


--- 1.60 seconds ---
done
3001
plot pcolormesh


...

--- 1.59 seconds ---
done
5001
plot pcolormesh


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)


--- 1.62 seconds ---
done
6001
plot pcolormesh


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)


--- 2.14 seconds ---
done


In [9]:
levels = [0, 1, 2, 5, 10, 15, 20, 25, 30, 40, 50, 60, 80, 100, 150, 200]
for r in range(0,51):
    for t in range(0,1):
        print(t*1000+r)
        prec = pr.isel(record=r,Time=t)
        plot(prec,r,0,t,' (mm/week)',levels)

50


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)
